In [ ]:
# import cv2
# import numpy as np
# import cv2 as cv
# from matplotlib import pyplot as plt
 
# img = cv2.imread(r'C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif')
# assert img is not None, "file could not be read, check with os.path.exists()"
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# cv2.imshow("Sure Background", img)
# cv2.waitKey(0)

In [41]:
!pip install opencv-python-headless numpy


Defaulting to user installation because normal site-packages is not writeable

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\cheukjy\\AppData\\Roaming\\Python\\Python311\\site-packages\\cv2\\cv2.pyd'
Check the permissions.




  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


In [ ]:
import cv2
import numpy as np

# Load the image
image_path = r'C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif'
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to create a binary image
binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Remove small noise
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

# Sure background area
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labelling
markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers + 1

# Mark the unknown region with zero
markers[unknown == 255] = 0

# Apply the watershed algorithm
markers = cv2.watershed(image, markers)

# Mark boundaries
image[markers == -1] = [0, 0, 255]

# Save or display the result
cv2.imwrite(r'C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif', image)


In [5]:
%matplotlib inline


In [11]:
pip install squidpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import squidpy as sq

# load the H&E stained tissue image and crop to a smaller segment
img = sq.datasets.visium_hne_image_crop()
crop = img.crop_corner(0, 0, size=1000)

# smooth image
sq.im.process(crop, layer=r'C:\Users\cheukjy\WC25_LLEDGE2_test\Test_2_H_tiles\267_29_H_test_2.tif', method="smooth", sigma=4)

# plot the result
fig, axes = plt.subplots(1, 2)
for layer, ax in zip(["image", "image_smooth"], axes):
    crop.show(layer, ax=ax)
    ax.set_title(layer)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
crop.show("image_smooth", cmap="gray", ax=axes[0])
axes[1].imshow(crop["image_smooth"][:, :, 0, 0] < 90)
_ = sns.histplot(np.array(crop["image_smooth"]).flatten(), bins=50, ax=axes[2])
plt.tight_layout()

sq.im.segment(img=crop, layer="image_smooth", method="watershed", thresh=90, geq=False)

print(crop)
print(f"Number of segments in crop: {len(np.unique(crop['segmented_watershed']))}")

fig, axes = plt.subplots(1, 2)
crop.show("image", channel=0, ax=axes[0])
_ = axes[0].set_title("H&E")
crop.show("segmented_watershed", cmap="jet", interpolation="none", ax=axes[1])
_ = axes[1].set_title("segmentation")